In [1]:
from PIL import ImageOps, Image, ImageGrab
#from IPython.display import Image, display 
from numpy import array
import numpy as np
import win32api

import time
import threading
import win32ui, win32gui, win32com, pythoncom, win32con
from win32com.client import Dispatch
from win32api import GetSystemMetrics

import winsound # beep

In [16]:
class Window:
    WINDOW_SUBSTRING = ['- the end -', 'Siebel Sales', 'Flask', 'Почта']
    URL = 'https://crmcorp.ca.sbrf.ru/crmcorp/start.swe?SWECmd=Start&SWEHo=crmcorp.ca.sbrf.ru'
    
    
    #def __init__(self):
        #self.window_info = {}
    
    def start_crm(self):
        ie = Dispatch("InternetExplorer.Application")
        ie.Visible = 1
        ie.Navigate(url)

    def get_window_info(self):
        window_info = {}
        #start_crm()
        win32gui.EnumWindows(self.set_window_coordinates, window_info)
        return window_info
    
    def set_window_coordinates(self, hwnd, window_info):
        # sets window coordinates
        if win32gui.IsWindowVisible(hwnd):
            for string in self.WINDOW_SUBSTRING:
                if string in win32gui.GetWindowText(hwnd):
                    #rect = win32gui.GetWindowRect(hwnd)          
                    rect = win32gui.GetWindowRect(win32gui.GetDesktopWindow())
                    x = rect[0]
                    y = rect[1]
                    w = rect[2] - x
                    h = rect[3] - y
                    window_info['name'] = win32gui.GetWindowText(hwnd)
                    win32com.client.Dispatch("WScript.Shell").SendKeys('%')
                    #start_crm()
                    win32gui.SetForegroundWindow(hwnd)
                    win32gui.SetWindowPos(hwnd, win32con.HWND_NOTOPMOST, x, y, w, h, 0)
                    
    def get_screen(self):
        time.sleep(3)
        size = win32gui.GetWindowRect(win32gui.GetDesktopWindow())
        screen = ImageGrab.grab(size)
        screen.save(r'.\screen\full.png', 'PNG')   
        img = array(screen.getdata(), dtype='int32').reshape((screen.size[1], screen.size[0], 3))
        return img

    def load_image(self, infilename):
        img = Image.open(infilename)
        img.load()
        data = np.asarray(img, dtype="int32")
        return data

    def save_image(self, npdata, outfilename):
        img = Image.fromarray(np.asarray(np.clip(npdata,0,255), dtype="uint32"), "L")
        img.save(outfilename)

    def click(self, arg):
        # click at position x, y
        win32api.SetCursorPos((arg[1],arg[0]))
        win32api.mouse_event(win32con.MOUSEEVENTF_LEFTDOWN,arg[1],arg[0],0,0) # press
        win32api.mouse_event(win32con.MOUSEEVENTF_LEFTUP,arg[1],arg[0],0,0) # up
        time.sleep(3)

    def finder(self, matrix, pattern):
        def comparer(r, c):
            if c + pattern.shape[1] > matrix.shape[1] or r + pattern.shape[0] > matrix.shape[0]:
                return
            if (np.subtract(matrix[r:r+pattern.shape[0], c:c+pattern.shape[1]], pattern) == 0).all():
                return True
            return
        def cmp():
            for row, sub in enumerate(matrix):
                for column, elem in enumerate(sub):
                    if (pattern[0][0] == elem).all(): # corner element
                        if comparer(row, column):
                            return row, column
        return cmp()

In [19]:
post = Window()
post.get_window_info()

try:
    post.click(post.finder(post.get_screen(), post.load_image(r'.\screen\make.png')))    
except:
    #winsound.Beep(2500, 200)
    print('No make')

try:
    post.click(post.finder(post.get_screen(), post.load_image(r'.\screen\send.png')))
    post.get_screen()
    #winsound.Beep(2000, 200)
except:
    print('No send')

try:
    if post.finder(post.get_screen(), post.load_image(r'.\screen\mistake.png')):
        post.click(post.finder(post.get_screen(), post.load_image(r'.\screen\undu.png')))
    post.get_screen()
    #winsound.Beep(2000, 200)
except:
    winsound.Beep(2500, 200)
winsound.Beep(2000, 200)

In [ ]:
# Retrieves information about the global cursor
# flags, hcursor, (x,y) = win32gui.GetCursorInfo()

In [ ]:
# import win32com.client # crl+a

# shell = win32com.client.Dispatch('WScript.Shell')
# shell.SendKeys('^a')